# Rover Response Optimization

This notebook shows how variables in an fmdtools model can be optimized for resilience.

In [1]:
from fmdtools.sim.sample import SampleApproach, FaultSample, FaultDomain, ParameterDomain
import fmdtools.analyze as an
import fmdtools.sim.propagate as prop
from fmdtools.sim.search import ParameterSimProblem
import matplotlib.pyplot as plt
import multiprocessing as mp
import time

Model is in defined rover_model.py

In [2]:
from examples.rover.rover_model import Rover, RoverParam

## Optimization

Here we define the optimization problem for the rover.

We use a parallel pool, staged execution, and minimal tracking options to lower computational cost as much as possible.

In [3]:
mdl = Rover(sp={'end_condition': ''})
track={'functions':{"Environment":"in_bound"},'flows':{"Ground":"all"}}
#rover_prob = search.ProblemInterface("rover_problem", mdl, pool=mp.Pool(5), staged=True, track=track)

In [4]:
mdl.p

RoverParam(ground=GroundParam(linetype='sine', amp=1.0, period=6.283185307179586, radius=20.0, x_start=10.0, y_end=10.0, x_min=0.0, x_max=30.0, x_res=0.1), correction=ResCorrection(ub_f=10.0, lb_f=-1.0, ub_t=10.0, lb_t=-1.0, ub_d=2.0, lb_d=-2.0, cor_d=1.0, cor_t=1.0, cor_f=1.0), degradation=DegParam(friction=0.0, drift=0.0), drive_modes={'mode_args': 'set'})

Here we will be optimizing over faults in the drive system at 3 points during the drive simulation interval:

In [5]:
endresults, mdlhist = prop.nominal(mdl)
phasemap = an.phases.from_hist(mdlhist)
fault_domain = FaultDomain(mdl)
fault_domain.add_all_fxnclass_modes('Drive')
fault_sample = FaultSample(fault_domain, phasemap['plan_path'])
fault_sample.add_fault_phases('drive',  args=(3,))

The variables are the correction factors int the fault management: 

In [6]:
pd = ParameterDomain(RoverParam)
pd.add_variables("correction.cor_f", "correction.cor_d", "correction.cor_t", lims={"correction.cor_f":(-10, 100), 
                                                                                         "correction.cor_d":(-100, 100),
                                                                                         "correction.cor_t":(-10, 100)})
pd

ParameterDomain with:
 - variables: {'correction.cor_f': (-10, 100), 'correction.cor_d': (-100, 100), 'correction.cor_t': (-10, 100)}
 - constants: {}
 - parameter_initializer: RoverParam

Now we setup the optimization problem using `ParameterSimProblem`

In [7]:
rover_prob = ParameterSimProblem(mdl, pd, 'fault_sample', fault_sample)
rover_prob

ParameterSimProblem with:
VARIABLES
 -correction.cor_f                                              nan
 -correction.cor_d                                              nan
 -correction.cor_t                                              nan

We can define multiple objectives, below we will use the end distanceand total_deviation from find_classification. 

In [8]:
rover_prob.add_result_objective('f1', 'endclass.end_dist')
rover_prob.add_result_objective('f2', 'endclass.tot_deviation')
rover_prob

ParameterSimProblem with:
VARIABLES
 -correction.cor_f                                              nan
 -correction.cor_d                                              nan
 -correction.cor_t                                              nan
OBJECTIVES
 -f1                                                            nan
 -f2                                                            nan

Here we do some basic timing:

In [9]:
rover_prob.f1(0.5,0.5,0.5)
a=time.time()
rover_prob.f1(0.6,0.5,0.5)
t=time.time()-a

Error at t=148.0 in scenario SingleFaultScenario(sequence={112.0: Injection(faults={'drive': ['hmode_7']}, disturbances={})}, times=(112.0,), function='drive', fault='hmode_7', rate=0.009523809523809523, name='drive_hmode_7_t112p0', time=112.0, phase='drive')


KeyboardInterrupt: 

In [ ]:
t

Alternatively, we can use:

In [10]:
rover_prob.time_execution(0.7, 0.5, 0.5)

16.926948070526123

In [11]:
rover_prob

ParameterSimProblem with:
VARIABLES
 -correction.cor_f                                           0.7000
 -correction.cor_d                                           0.5000
 -correction.cor_t                                           0.5000
OBJECTIVES
 -f1                                                      1186.1668
 -f2                                                        17.1813

In [18]:
dir(rover_prob)


ParameterDomain with:
 - variables: {'correction.cor_f': (-10, 100), 'correction.cor_d': (-100, 100), 'correction.cor_t': (-10, 100)}
 - constants: {}
 - parameter_initializer: RoverParam

### Rover Optimization: 

In [75]:
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch
from pymoo.problems.functional import FunctionalProblem
import numpy as np

In [76]:
pymoo_prob = rover_prob.to_pymoo_problem(objectives="end_dist")

AttributeError: 'ParameterSimProblem' object has no attribute 'to_pymoo_problem'

In [77]:
arg={}
pymoo_prob._evaluate([1,1,2],arg)

AttributeError: 'ParameterSimProblem' object has no attribute '_evaluate'

In [18]:
arg

In [19]:
algorithm=PatternSearch(x0=np.array([0,0,0])) 

In [31]:
res = minimize(pymoo_prob, algorithm, verbose=True)

In [32]:
res.X

### Results visualization

Here we look at the optimized results and compare with the starting results:

In [34]:
rover_prob.end_dist([   0.65790489, -100.        ,   -1.45828247])

In [35]:
fig = plot_trajectories(rover_prob._sims['drive_faults']['mdlhists'], app= app_drive)

In [39]:
an.plot.mdlhists(rover_prob._sims['drive_faults']['mdlhists'], fxnflowvals="Ground", time_slice=app_drive.times)

In [40]:
ecs = {k: v['endclass'] for k, v in rover_prob._sims['drive_faults']['results'].items()}
fig = an.plot.metric_dist(ecs, 'end_dist')

starting solution:

In [41]:
rover_prob.end_dist([0,0,0])

In [42]:
fig = plot_trajectories(rover_prob._sims['drive_faults']['mdlhists'], app= app_drive)

In [43]:
ecs = {k: v['endclass'] for k, v in rover_prob._sims['drive_faults']['results'].items()}
fig = an.plot.metric_dist(ecs, 'end_dist')

As shown, while the optimized correction factors don't mitigate all scenarios, they do increase the number of scenarios that are mitigated.